In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [2]:
url = "https://www.vlr.gg/event/stats/2283/valorant-champions-2025"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}

In [3]:
try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()
except requests.exceptions.RequestException as e:
    print(f"Erro ao acessar a URL: {e}")
    exit()


In [4]:
soup = BeautifulSoup(response.content, 'html.parser')
stats_table = soup.find('table', class_='wf-table mod-stats mod-scroll')

In [ ]:
if stats_table:
    table_headers = [header.get_text(strip=True) for header in stats_table.find_all('th')]
    
    rows = []
    for row in stats_table.find('tbody').find_all('tr'):
        # Extrai o nome do jogador e time juntos
        player_cell = row.find('td', class_='mod-player')
        player_name_div = player_cell.find('div', class_='text-of')
        player_team_div = player_cell.find('div', class_='stats-player-country')
        
        # Garante que estamos pegando os textos corretos
        player = player_name_div.get_text(strip=True) if player_name_div else ''
        team = player_team_div.get_text(strip=True) if player_team_div else ''
        
        # Constrói a linha de dados
        row_data = [(player + ' ' + team).strip()] # Combina para manter a estrutura inicial
        
        # Pega o resto das células da linha
        other_cells = row.find_all('td')[1:]  # Ignora a primeira célula que já foi processada
        for cell in other_cells:
            # Tratamento especial para células com spans (como ACS, K/D, etc)
            span_val = cell.find('span', class_='stats-sq')
            if span_val:
                row_data.append(span_val.get_text(strip=True))
            else:
                row_data.append(cell.get_text(strip=True))

        rows.append(row_data)

In [6]:
df = pd.DataFrame(rows, columns=table_headers)
df_anterior = df.copy()  # Cópia do DataFrame antes das modificações

In [7]:
df_anterior.head(10)

,Player,Agents,Rnd,R2.0,ACS,K:D,KAST,ADR,KPR,APR,...,FDPR,HS%,CL%,CL,KMax,K,D,A,FK,FD
0,something PRX,,106,1.39,267.2,1.61,79%,166.1,0.97,0.37,...,0.08,26%,23%,3/13,23,103,64,39,18,9
1,RieNs TH,,93,1.31,249.0,1.41,80%,162.1,0.89,0.35,...,0.03,35%,15%,2/13,23,83,59,33,5,3
2,SpiritZ1 DRG,,58,1.31,274.5,1.20,86%,179.1,0.91,0.41,...,0.07,31%,,0/1,27,53,44,24,13,4
3,aspas MIBR,(+1),98,1.26,255.2,1.48,78%,167.7,0.91,0.17,...,0.07,25%,,0/8,22,89,60,17,19,7
4,Flashback DRX,,109,1.25,258.4,1.41,72%,170.1,0.94,0.17,...,0.15,26%,11%,1/9,26,103,73,19,21,16
5,Chronicle FNC,,104,1.24,210.4,1.30,80%,149.2,0.76,0.31,...,0.05,30%,6%,1/18,25,79,61,32,11,5
6,Kushy RRQ,,43,1.21,254.0,1.19,67%,175.0,0.86,0.21,...,0.12,49%,25%,1/4,20,37,31,9,6,5
7,Jinggg PRX,,106,1.21,257.0,1.32,78%,162.2,0.89,0.24,...,0.13,22%,,0/5,24,94,71,25,15,14
8,johnqt SEN,,97,1.19,191.0,1.16,82%,116.8,0.67,0.29,...,0.04,31%,,0/12,17,65,56,28,5,4
9,Nicc DRG,,58,1.18,227.0,1.32,83%,142.3,0.84,0.26,...,0.02,34%,,0/11,25,49,37,15,2,1


In [8]:
df[['Player', 'Team']] = df['Player'].str.rsplit(' ', n=1, expand=True)

In [ ]:
if 'KAST' in df.columns:
        df['KAST'] = df['KAST'].str.replace('%', '').astype(float) / 100.0

if 'HS%' in df.columns:
        df['HS%'] = df['HS%'].str.replace('%', '').astype(float) / 100.0

if 'CL%' in df.columns:
        df['CL%'][df['CL%'] == ''] = '0%'
        df['CL%'] = df['CL%'].str.replace('%', '').astype(float) / 100.0

if 'CL' in df.columns:
        df['CL'][df['CL'] == ''] = '0/0'


In [10]:
cols_to_convert = ['R', 'ACS', 'K:D', 'ADR',
                    'KPR', 'APR', 'FKPR', 'FDPR',
                      'K', 'D', 'A', 'FK',
                        'FD', '+/-', 'KMax', 'Rnd', 'R2.0']
    
for col in cols_to_convert:
    if col in df.columns:
        # pd.to_numeric converte para número. errors='coerce' transforma o que não for número em NaN (vazio)
        df[col] = pd.to_numeric(df[col], errors='coerce')


In [ ]:
# 4. Reordenar as colunas para melhor visualização
# Colocando 'Player' e 'Team' no início
all_cols = df.columns.tolist()
all_cols.remove('Player')
all_cols.remove('Team')
new_order = ['Player', 'Team'] + all_cols
df = df[new_order]

In [12]:
df = df.drop(columns=['Agents'])

In [13]:
df.head(10)

,Player,Team,Rnd,R2.0,ACS,K:D,KAST,ADR,KPR,APR,...,FDPR,HS%,CL%,CL,KMax,K,D,A,FK,FD
0,something,PRX,106,1.39,267.2,1.61,0.79,166.1,0.97,0.37,...,0.08,0.26,0.23,3/13,23,103,64,39,18,9
1,RieNs,TH,93,1.31,249.0,1.41,0.80,162.1,0.89,0.35,...,0.03,0.35,0.15,2/13,23,83,59,33,5,3
2,SpiritZ1,DRG,58,1.31,274.5,1.20,0.86,179.1,0.91,0.41,...,0.07,0.31,0.00,0/1,27,53,44,24,13,4
3,aspas,MIBR,98,1.26,255.2,1.48,0.78,167.7,0.91,0.17,...,0.07,0.25,0.00,0/8,22,89,60,17,19,7
4,Flashback,DRX,109,1.25,258.4,1.41,0.72,170.1,0.94,0.17,...,0.15,0.26,0.11,1/9,26,103,73,19,21,16
5,Chronicle,FNC,104,1.24,210.4,1.30,0.80,149.2,0.76,0.31,...,0.05,0.30,0.06,1/18,25,79,61,32,11,5
6,Kushy,RRQ,43,1.21,254.0,1.19,0.67,175.0,0.86,0.21,...,0.12,0.49,0.25,1/4,20,37,31,9,6,5
7,Jinggg,PRX,106,1.21,257.0,1.32,0.78,162.2,0.89,0.24,...,0.13,0.22,0.00,0/5,24,94,71,25,15,14
8,johnqt,SEN,97,1.19,191.0,1.16,0.82,116.8,0.67,0.29,...,0.04,0.31,0.00,0/12,17,65,56,28,5,4
9,Nicc,DRG,58,1.18,227.0,1.32,0.83,142.3,0.84,0.26,...,0.02,0.34,0.00,0/11,25,49,37,15,2,1


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  80 non-null     object 
 1   Team    80 non-null     object 
 2   Rnd     80 non-null     int64  
 3   R2.0    80 non-null     float64
 4   ACS     80 non-null     float64
 5   K:D     80 non-null     float64
 6   KAST    80 non-null     float64
 7   ADR     80 non-null     float64
 8   KPR     80 non-null     float64
 9   APR     80 non-null     float64
 10  FKPR    80 non-null     float64
 11  FDPR    80 non-null     float64
 12  HS%     80 non-null     float64
 13  CL%     80 non-null     float64
 14  CL      80 non-null     object 
 15  KMax    80 non-null     int64  
 16  K       80 non-null     int64  
 17  D       80 non-null     int64  
 18  A       80 non-null     int64  
 19  FK      80 non-null     int64  
 20  FD      80 non-null     int64  
dtypes: float64(11), int64(7), object(3)
memor

In [15]:
df_anterior.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Player  80 non-null     object
 1   Agents  80 non-null     object
 2   Rnd     80 non-null     object
 3   R2.0    80 non-null     object
 4   ACS     80 non-null     object
 5   K:D     80 non-null     object
 6   KAST    80 non-null     object
 7   ADR     80 non-null     object
 8   KPR     80 non-null     object
 9   APR     80 non-null     object
 10  FKPR    80 non-null     object
 11  FDPR    80 non-null     object
 12  HS%     80 non-null     object
 13  CL%     80 non-null     object
 14  CL      80 non-null     object
 15  KMax    80 non-null     object
 16  K       80 non-null     object
 17  D       80 non-null     object
 18  A       80 non-null     object
 19  FK      80 non-null     object
 20  FD      80 non-null     object
dtypes: object(21)
memory usage: 13.3+ KB


In [16]:
df_anterior.head(10)

,Player,Agents,Rnd,R2.0,ACS,K:D,KAST,ADR,KPR,APR,...,FDPR,HS%,CL%,CL,KMax,K,D,A,FK,FD
0,something PRX,,106,1.39,267.2,1.61,79%,166.1,0.97,0.37,...,0.08,26%,23%,3/13,23,103,64,39,18,9
1,RieNs TH,,93,1.31,249.0,1.41,80%,162.1,0.89,0.35,...,0.03,35%,15%,2/13,23,83,59,33,5,3
2,SpiritZ1 DRG,,58,1.31,274.5,1.20,86%,179.1,0.91,0.41,...,0.07,31%,,0/1,27,53,44,24,13,4
3,aspas MIBR,(+1),98,1.26,255.2,1.48,78%,167.7,0.91,0.17,...,0.07,25%,,0/8,22,89,60,17,19,7
4,Flashback DRX,,109,1.25,258.4,1.41,72%,170.1,0.94,0.17,...,0.15,26%,11%,1/9,26,103,73,19,21,16
5,Chronicle FNC,,104,1.24,210.4,1.30,80%,149.2,0.76,0.31,...,0.05,30%,6%,1/18,25,79,61,32,11,5
6,Kushy RRQ,,43,1.21,254.0,1.19,67%,175.0,0.86,0.21,...,0.12,49%,25%,1/4,20,37,31,9,6,5
7,Jinggg PRX,,106,1.21,257.0,1.32,78%,162.2,0.89,0.24,...,0.13,22%,,0/5,24,94,71,25,15,14
8,johnqt SEN,,97,1.19,191.0,1.16,82%,116.8,0.67,0.29,...,0.04,31%,,0/12,17,65,56,28,5,4
9,Nicc DRG,,58,1.18,227.0,1.32,83%,142.3,0.84,0.26,...,0.02,34%,,0/11,25,49,37,15,2,1


In [17]:
df.to_csv('valorant_champions_2025_stats.csv', index=False)